In [1]:
import pandas as pd
import numpy as np

In [2]:
mydir = "/Users/samhsu/Documents/THESIS/Out/"
#wrist = pd.read_csv(mydir + "Actigraph_complete23.csv")
biostamp = pd.read_csv(mydir + "Biostamp_complete.csv")

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,thigh.mean.vm,thigh.sd.vm,thigh.mean.ang,thigh.sd.ang,thigh.p625,thigh.dfreq,thigh.ratio.df,...,secondary_behavior,secondary_posture,secondary_upperbody,secondary_intensity,num_postures,transition,actual_time,posture_coding,type,posture_coding_ordinal


In [3]:
def create_tier1_column(ground_truth):
    stand = ((ground_truth['primary_posture'] == 'LA- stand and move') 
             | (ground_truth['primary_posture'] == 'LA- stand and move with upper body movement') 
             | (ground_truth['primary_posture'] == 'LA- stand and move with unidentifiable upper body movement'))
    walk = ((ground_truth['primary_posture'] == 'WA-walk with load') 
            | (ground_truth['primary_posture'] == 'WA- ascend stairs')
            | (ground_truth['primary_posture'] == 'WA- descend stairs')
            | (ground_truth['primary_posture'] == 'WA- running'))
    sitting = ((ground_truth['primary_posture'] == 'LA- kneeling/ squatting') 
               | (ground_truth['primary_posture'] == 'LA- stretching')) & (ground_truth['primary_intensity'] == 'sedentary')
    
    ground_truth['posture1'] = np.where(stand, 'LA- stand',
                                       np.where(walk, 'WA- walk',
                                               np.where(sitting, 'SB-sitting',
                                                       ground_truth['primary_posture'])))
    ground_truth.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
    return ground_truth

In [4]:
def create_tier2_column(ground_truth):
    # absorb all types of "LA - stand and move" together - don't care about intensity
    la = (ground_truth['primary_posture'] == 'LA- stand and move') | (ground_truth['primary_posture'] == 'LA- stand and move with upper body movement') | (ground_truth['primary_posture'] == 'LA- stand and move with unidentifiable upper body movement')

    # absorb walk and walk w/ load
    wa = (ground_truth['primary_posture'] == 'WA- walk') | (ground_truth['primary_posture'] == 'WA-walk with load')

    # absorb "LA - kneeling/squatting" and "LA - stretching"
    kneel_squat_stretch = (ground_truth['primary_posture'] == 'LA- kneeling/ squatting') | (ground_truth['primary_posture'] == 'LA- stretching')

    # absorb "LA - kneeling/squatting"/"LA-stretching" into "SB-sitting" if sedentary
    sed_kneel_squat_stretch = kneel_squat_stretch & (ground_truth['primary_intensity'] == 'sedentary')


    ground_truth['posture2'] = np.where(la, 'LA- stand and move',
                                            np.where(wa, 'WA- walk',
                                                    np.where(sed_kneel_squat_stretch, 'SB-sitting',
                                                             ground_truth['primary_posture'])))
    
    return ground_truth

In [4]:
def create_MET_column(ground_truth):
    sed = ((ground_truth['primary_posture'] == 'SB-sitting') | 
           (ground_truth['primary_posture'] == 'SB- lying') |
           (ground_truth['primary_posture'] == 'LA- kneeling/ squatting'))
    light = ((ground_truth['primary_posture'] == 'LA- stand')
             | (ground_truth['primary_posture'] == 'LA- stand and move') 
             | (ground_truth['primary_posture'] == 'LA- stand and move with upper body movement') 
             | (ground_truth['primary_posture'] == 'LA- stand and move with unidentifiable upper body movement'))
    moderate = ((ground_truth['primary_posture'] == 'WA- walk')
                | (ground_truth['primary_posture'] == 'WA-walk with load') 
                | (ground_truth['primary_posture'] == 'WA- ascend stairs')
                | (ground_truth['primary_posture'] == 'WA- descend stairs'))
    vigorous = ((ground_truth['primary_posture'] == 'WA- running')
               | (ground_truth['primary_posture'] == 'SP- bike')
               | (ground_truth['primary_posture'] == 'SP- other sport movement'))
    
    ground_truth['MET_coding'] == np.where(sed, 'sedentary',
                                          np.where(light, 'light',
                                                  np.where(moderate, 'moderate',
                                                          np.where(vigorous, 'vigorous',
                                                                  ground_truth['primary_posture']))))
    
    return ground_truth

In [5]:
def create_Ellis_column(ground_truth):
    driving = ((ground_truth['primary_behavior'] == 'TRAV- passenger (Car/truck/motorcycle)') |
               (ground_truth['primary_behavior'] == 'TRAV- driver (Car/truck/motorcycle)') |
               (ground_truth['primary_behavior'] == 'TRAV- passenger bus or train'))
    sit = ((ground_truth['primary_posture'] == 'SB-sitting') | 
           (ground_truth['primary_posture'] == 'SB- lying') |
           (ground_truth['primary_posture'] == 'LA- kneeling/ squatting'))
    standing = ((ground_truth['primary_posture'] == 'LA- stand')
             | (ground_truth['primary_posture'] == 'LA- stand and move') 
             | (ground_truth['primary_posture'] == 'LA- stand and move with upper body movement') 
             | (ground_truth['primary_posture'] == 'LA- stand and move with unidentifiable upper body movement'))
    walking = ((ground_truth['primary_posture'] == 'WA- walk')
                | (ground_truth['primary_posture'] == 'WA-walk with load') 
                | (ground_truth['primary_posture'] == 'WA- ascend stairs')
                | (ground_truth['primary_posture'] == 'WA- descend stairs')
                | (ground_truth['primary_posture'] == 'WA- running'))
    other = ((ground_truth['primary_posture'] == 'SP- bike')
               | (ground_truth['primary_posture'] == 'SP- other sport movement'))
    
    ground_truth['Ellis_coding'] = np.where(driving, 'riding in vehicle',
                                           np.where(sit, 'sitting',
                                                   np.where(standing, 'standing',
                                                           np.where(walking, 'walking/running',
                                                                   np.where(other, 'other',
                                                                           ground_truth['primary_posture'])))))
        
               

In [7]:
list(biostamp)

['Unnamed: 0',
 'Unnamed: 0.1',
 'Unnamed: 0.1.1',
 'thigh.mean.vm',
 'thigh.sd.vm',
 'thigh.mean.ang',
 'thigh.sd.ang',
 'thigh.p625',
 'thigh.dfreq',
 'thigh.ratio.df',
 'thigh.min.x',
 'thigh.min.y',
 'thigh.min.z',
 'thigh.max.x',
 'thigh.max.y',
 'thigh.max.z',
 'thigh.mean.x',
 'thigh.mean.y',
 'thigh.mean.z',
 'thigh.sd.x',
 'thigh.sd.y',
 'thigh.sd.z',
 'thigh.mean.xy',
 'thigh.mean.yz',
 'thigh.mean.xz',
 'thigh.mean.xyz',
 'start.time',
 'chest.mean.vm',
 'chest.sd.vm',
 'chest.mean.ang',
 'chest.sd.ang',
 'chest.p625',
 'chest.dfreq',
 'chest.ratio.df',
 'chest.min.x',
 'chest.min.y',
 'chest.min.z',
 'chest.max.x',
 'chest.max.y',
 'chest.max.z',
 'chest.mean.x',
 'chest.mean.y',
 'chest.mean.z',
 'chest.sd.x',
 'chest.sd.y',
 'chest.sd.z',
 'chest.mean.xy',
 'chest.mean.yz',
 'chest.mean.xz',
 'chest.mean.xyz',
 'coding',
 'primary_behavior',
 'primary_posture',
 'primary_upperbody',
 'primary_intensity',
 'secondary_behavior',
 'secondary_posture',
 'secondary_upperbody',

In [6]:
wrist = create_tier2_column(wrist)

In [34]:
biostamp = create_tier1_column(biostamp)
biostamp = create_tier2_column(biostamp)

In [7]:
wrist.to_csv("tier_wrist_complete23.csv")
#biostamp.to_csv("tier_biostamp_complete.csv")

In [38]:
biostamp.shape

(354912, 65)